In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/buildings-dataset/best_net.pth
/kaggle/input/buildings-dataset/best_fcn.pth
/kaggle/input/buildings-dataset/label/T2209.png
/kaggle/input/buildings-dataset/label/W949.png
/kaggle/input/buildings-dataset/label/T4089.png
/kaggle/input/buildings-dataset/label/T3521.png
/kaggle/input/buildings-dataset/label/T3618.png
/kaggle/input/buildings-dataset/label/T721.png
/kaggle/input/buildings-dataset/label/T1189.png
/kaggle/input/buildings-dataset/label/W752.png
/kaggle/input/buildings-dataset/label/T2686.png
/kaggle/input/buildings-dataset/label/T4111.png
/kaggle/input/buildings-dataset/label/T2241.png
/kaggle/input/buildings-dataset/label/T4114.png
/kaggle/input/buildings-dataset/label/W346.png
/kaggle/input/buildings-dataset/label/T162.png
/kaggle/input/buildings-dataset/label/T2533.png
/kaggle/input/buildings-dataset/label/T794.png
/kaggle/input/buildings-dataset/label/W1443.png
/kaggle/input/buildings-dataset/label/S005.png
/kaggle/input/buildings-dataset/label/W1362.png
/kagg

In [2]:
import os
from torch import nn
import torch
import time

from PIL import Image
from torchvision import transforms

from torch.utils.data import DataLoader, Dataset
from torch.optim import Adam, AdamW, SGD
import torch.nn.functional as F

In [3]:
import os
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image

class ImageSegmentation(Dataset):
    def __init__(self, img_dir, mask_dir):
        self.img_dir = img_dir
        self.mask_dir = mask_dir

        self.img_transform = transforms.Compose([
            transforms.Resize((512, 512)),
            transforms.ToTensor()
        ])
        self.mask_transform = transforms.Compose([
            transforms.Resize((512, 512), interpolation=Image.NEAREST),
            transforms.ToTensor()
        ])

        valid_extensions = {".jpg", ".jpeg", ".png"}
        self.images = [f for f in os.listdir(img_dir) if os.path.splitext(f)[1].lower() in valid_extensions]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_path = os.path.join(self.img_dir, self.images[idx])
        name, _ = os.path.splitext(self.images[idx])
        mask_path = os.path.join(self.mask_dir, f"{name}.png")

        image = Image.open(image_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")

        image = self.img_transform(image)
        mask = self.mask_transform(mask)

        mask = (mask > 0.5).float()

        return image, mask


In [4]:
def get_dataloader(img_dir, mask_dir, batch_size=2, shuffle=True):
    dataset = ImageSegmentation(img_dir, mask_dir)
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

In [5]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv_op = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.conv_op(x)


In [6]:
class DownSample(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = DoubleConv(in_channels, out_channels)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        down = self.conv(x)
        p = self.pool(down)
        return down, p

In [7]:

class UpSample(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
        self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)

        # Pad in case of odd dimensions mismatch
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]
        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])

        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class UNet(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        self.down_convolution_1 = DownSample(in_channels, 64)
        self.down_convolution_2 = DownSample(64, 128)
        self.down_convolution_3 = DownSample(128, 256)
        self.down_convolution_4 = DownSample(256, 512)

        self.bottleneck = DoubleConv(512, 1024)

        self.up_convolution_1 = UpSample(1024, 512)
        self.up_convolution_2 = UpSample(512, 256)
        self.up_convolution_3 = UpSample(256, 128)
        self.up_convolution_4 = UpSample(128, 64)

        # final output conv
        self.out = nn.Conv2d(64, num_classes, kernel_size=1)

    def forward(self, x):
        down_1, p1 = self.down_convolution_1(x)
        down_2, p2 = self.down_convolution_2(p1)
        down_3, p3 = self.down_convolution_3(p2)
        down_4, p4 = self.down_convolution_4(p3)

        b = self.bottleneck(p4)

        up_1 = self.up_convolution_1(b, down_4)
        up_2 = self.up_convolution_2(up_1, down_3)
        up_3 = self.up_convolution_3(up_2, down_2)
        up_4 = self.up_convolution_4(up_3, down_1)

        out = self.out(up_4)

        return out   # logits (apply sigmoid/softmax outside depending on loss)


In [9]:
class DiceLoss(nn.Module):
    def __init__(self, smooth=1e-6):
        super(DiceLoss, self).__init__()
        self.smooth=smooth

    def forward(self, inputs, targets):
        inputs=inputs.view(-1)
        targets=targets.view(-1)

        intersection=(inputs * targets).sum()
        dice_score=(2. * intersection + self.smooth) / (inputs.sum() + targets.sum() + self.smooth)

        return 1 - dice_score

In [10]:
class BCEWithDiceLoss(nn.Module):
    def __init__(self, smooth=1e-6):
        super(BCEWithDiceLoss, self).__init__()
        self.bce = nn.BCEWithLogitsLoss()
        self.dice = DiceLoss()

    def forward(self, inputs, targets):
        bce_loss = self.bce(inputs, targets)
        dice_loss = self.dice(inputs, targets)
        return 0.5*bce_loss + dice_loss
        

In [11]:
import torch
import os
from torch.cuda.amp import autocast, GradScaler
from torch.optim import Adam  # you can use SGD if you prefer

def train(model, dataloader, epochs=10, lr=0.001, save_path="unet_model", load_path=None):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load checkpoint if available
    if load_path and os.path.exists(load_path):
        print(f"Loading model weights from {load_path}")
        model.load_state_dict(torch.load(load_path, map_location=device))
    else:
        print("No checkpoint found, training from scratch")

    print(f"Using device: {device}")
    model.to(device)

    # Loss & optimizer
    criterion = BCEWithDiceLoss()   # keep your custom loss
    optimizer = Adam(model.parameters(), lr=lr)
    scaler = GradScaler()  # for mixed precision

    for epoch in range(epochs):
        model.train()
        epoch_loss = 0.0

        for images, mask in dataloader:
            images, mask = images.to(device), mask.to(device)

            optimizer.zero_grad()

            # mixed precision training
            with autocast():
                output = model(images)
                loss = criterion(output, mask)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            epoch_loss += loss.item()

        avg_loss = epoch_loss / len(dataloader)
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}, LR: {lr}")

        # save every 10 epochs
        if (epoch + 1) % 10 == 0:
            torch.save(model.state_dict(), f"{save_path}_epoch{epoch+1}.pth")

        # free cached GPU memory
        torch.cuda.empty_cache()

    # final save
    torch.save(model.state_dict(), f"{save_path}_final.pth")
    print(f"Model saved to {save_path}_final.pth")


In [12]:
train_dataset = ImageSegmentation(
    img_dir="/kaggle/input/buildings-dataset/src",
    mask_dir="/kaggle/input/buildings-dataset/label"
)

In [13]:
dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)

In [14]:
model = UNet(in_channels=3, num_classes=1)
train(model, dataloader, epochs=1, lr=0.001)

No checkpoint found, training from scratch
Using device: cuda


/tmp/ipykernel_19/454041104.py:22: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()  # for mixed precision
/tmp/ipykernel_19/454041104.py:34: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch [1/1], Loss: 1.2850, LR: 0.001
Model saved to unet_model_final.pth


In [15]:
dataloader = get_dataloader("/kaggle/input/buildings-dataset/src", "/kaggle/input/buildings-dataset/label", batch_size=1, shuffle=True)

In [16]:
model=UNet(in_channels=3, num_classes=1)

In [17]:
train(model, dataloader, epochs=1, lr=0.001)

No checkpoint found, training from scratch
Using device: cuda


/tmp/ipykernel_19/454041104.py:22: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()  # for mixed precision
/tmp/ipykernel_19/454041104.py:34: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch [1/1], Loss: nan, LR: 0.001
Model saved to unet_model_final.pth
